#Importación de librerías, url y demás datos para el uso en el demo

In [ ]:
from matplotlib.pyplot import plot as plt



#url =https://raw.githubusercontent.com/myriammmm13/demoDiscrestas/refs/heads/main/clothes_price_prediction_data.csv


ModuleNotFoundError: No module named 'torch'